### Seta as configurações de paralelismo

In [0]:
from pyspark.sql.functions import col, when, to_date, lit, broadcast, asc, desc, date_format, sum, coalesce, date_trunc, monotonically_increasing_id, round

In [0]:
spark.conf.set('spark.sql.shuffle.partitions', spark.sparkContext.defaultParallelism)

### Seta as configurações data e hora

In [0]:
import os, time
from datetime import datetime
os.environ['TZ'] = 'America/Sao_Paulo'
time.tzset()
spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")
inicio_etl = datetime.now()

### Cria as tabelas Spark SQL
> Apenas quando há tabelas arquivos parquet novos (atualizados).

##### PRODDG

In [0]:
def escreveTabelas(nm_tabela):
  nm_tabela = nm_tabela.replace('.parquet', '')
  if "dw_dri" not in nm_tabela:
    colunas = spark.read.parquet(f'/FileStore/tables/{nm_tabela}.parquet').columns
    spark.read.parquet(f'/FileStore/tables/{nm_tabela}.parquet')\
      .sort(colunas[0])\
      .repartition(spark.sparkContext.defaultParallelism, colunas[0])\
      .write\
      .mode('overwrite')\
      .saveAsTable(f'proddg.{nm_tabela}')
    dbutils.fs.rm(f'/FileStore/tables/{nm_tabela}.parquet')
  else:
    colunas = spark.read.parquet(f'/FileStore/tables/{nm_tabela}.parquet').columns
    spark.read.parquet(f'/FileStore/tables/{nm_tabela}.parquet')\
      .sort(colunas[0])\
      .repartition(spark.sparkContext.defaultParallelism, colunas[0])\
      .write\
      .mode('overwrite')\
      .saveAsTable(f'dw_dri.{nm_tabela}')
    dbutils.fs.rm(f'/FileStore/tables/{nm_tabela}.parquet')

In [0]:
spark.conf.set('spark.sql.legacy.parquet.datetimeRebaseModeInRead', 'LEGACY')
spark.conf.set('spark.sql.legacy.parquet.int96RebaseModeInWrite', 'LEGACY')
[escreveTabelas(i.name) for i in dbutils.fs.ls('/FileStore/tables/')]

Out[5]: []

In [0]:
def tabelaColuna(df, nm_opt=None):
  '''Output em texto para renomear as colunas tabela_coluna'''
  if nm_opt == None:
    strDf = str(getVariableName(df))
  else:
    strDf = str(nm_opt)
  colunas = df.columns
  for i in colunas:
    print(f""".withColumnRenamed('{i}', '{strDf+i}') \\""")  

### PAGAMENTO

In [0]:
dfProcessoCronogramaFinanceiro = spark.table('proddg.PROCESSO_CRONOGRAMA_FINANCEIRO')
dfProcessoParcelaCronograma = spark.table('proddg.PROCESSO_PARCELA_CRONOGRAMA')
dfScbaRubricaFichaFinanceira = spark.table('proddg.SCBA_RUBRICA_FICHA_FINANCEIRA')

In [0]:
SQ_PARCELA_CRONOGRAMA_05 = \
dfProcessoCronogramaFinanceiro\
  .join(dfProcessoParcelaCronograma, 'ID_CRONOGRAMA_FINANCEIRO', 'inner')\
  .join(dfScbaRubricaFichaFinanceira, 'ID_PARCELA_CRONOGRAMA', 'leftouter')\
  .filter(col('TP_SITUACAO_PARCELA').isin(1, 2)) \
  .filter(col('ID_RUBRICA_FICHA_FINANCEIRA').isNull()) \
  .select('ID_PROCESSO', 'DT_EMPENHO_SIAFI', 'DT_ORDEM_BANCARIA', 'TP_SITUACAO_PARCELA', 'TP_DESPESA', 'AN_MES_EMPENHO', 'AN_MES_PAGAMENTO', 'VL_PAGAMENTO', 'NR_EMPENHO', 'NR_ORDEM_BANCARIA', 'ID_MOEDA_PAGAMENTO')\
  .cache()\
  .filter(col('TP_SITUACAO_PARCELA')==1) \
  .withColumn('DT_REFERENCIA_EMPENHO', to_date(col('AN_MES_EMPENHO'), 'yyyyMM'))\
  .withColumn('DT_REFERENCIA_PAGAMENTO', to_date(col('AN_MES_PAGAMENTO'), 'yyyyMM'))\
  .withColumn('DT_ORDEM_BANCARIA', col('DT_ORDEM_BANCARIA').cast('date'))\
  .withColumn('DT_EMPENHO', when(col('DT_EMPENHO_SIAFI').isNull(), col('DT_REFERENCIA_EMPENHO')).otherwise(col('DT_EMPENHO_SIAFI')).cast('date'))\
  .withColumn('DT_PAGAMENTO', when(col('DT_ORDEM_BANCARIA').isNull(), col('DT_REFERENCIA_PAGAMENTO')).otherwise(col('DT_ORDEM_BANCARIA')).cast('date'))\
  .withColumn('ID_RUBRICA', when(col('TP_DESPESA')==1, 54).when(col('TP_DESPESA')==2, 55))\
  .withColumnRenamed('VL_PAGAMENTO', 'VL_RUBRICA')\
  .withColumn('TP_SOLICITACAO_PAGAMENTO', lit(1))\
  .withColumn('TP_SOLICITACAO_EMPENHO', lit(0))\
  .withColumn('TP_BENEFICIO', lit(0))\
  .drop('DT_EMPENHO_SIAFI', 'TP_DESPESA', 'AN_MES_EMPENHO', 'AN_MES_PAGAMENTO')
#display(SQ_PARCELA_CRONOGRAMA_05)

##### Empenho AUXPE

In [0]:
FIL_EMPENHO_01 = \
  SQ_PARCELA_CRONOGRAMA_05\
  .filter(col('TP_SITUACAO_PARCELA').isin(1, 2))\
  .withColumn('TP_SOLICITACAO', lit(0))\
  .withColumnRenamed('NR_EMPENHO', 'NR_SOLICITACAO')\
  .withColumnRenamed('DT_EMPENHO', 'DT_PROCESSAMENTO')\
  .withColumnRenamed('DT_REFERENCIA_EMPENHO', 'DT_REFERENCIA')\
  .select('ID_PROCESSO', 'ID_RUBRICA', 'TP_SOLICITACAO', 'NR_SOLICITACAO', 'DT_PROCESSAMENTO', 'DT_REFERENCIA', 'VL_RUBRICA', 'TP_SITUACAO_PARCELA', 'TP_BENEFICIO', 'ID_MOEDA_PAGAMENTO') 
#display(FIL_EMPENHO_01)

##### Pagamento AUXPE

In [0]:
FIL_PAGAMENTO_02 = \
  SQ_PARCELA_CRONOGRAMA_05\
  .filter(col('TP_SITUACAO_PARCELA')==1)\
  .withColumn('TP_SOLICITACAO', lit(1))\
  .withColumnRenamed('NR_ORDEM_BANCARIA', 'NR_SOLICITACAO')\
  .withColumnRenamed('DT_PAGAMENTO', 'DT_PROCESSAMENTO')\
  .withColumnRenamed('DT_REFERENCIA_PAGAMENTO', 'DT_REFERENCIA')\
  .select('ID_PROCESSO', 'ID_RUBRICA', 'TP_SOLICITACAO', 'NR_SOLICITACAO', 'DT_PROCESSAMENTO', 'DT_REFERENCIA', 'VL_RUBRICA', 'TP_SITUACAO_PARCELA', 'TP_BENEFICIO', 'ID_MOEDA_PAGAMENTO') 
#display(FIL_PAGAMENTO_02)

##### Ficha Financeira

In [0]:
dfScbaFichaFinanceira = spark.table('proddg.SCBA_FICHA_FINANCEIRA')
dfScbaFolhaPagamento = spark.table('proddg.SCBA_FOLHA_PAGAMENTO')
dfProcessoProcesso = spark.table('proddg.PROCESSO_PROCESSO')

In [0]:
SQ_FICHA_FINANCEIRA_02 = \
dfScbaFichaFinanceira \
  .join(dfScbaFolhaPagamento, 'ID_FOLHA_PAGAMENTO', 'inner')\
  .join(dfProcessoProcesso, 'ID_PROCESSO', 'inner')\
  .filter(col('TP_BENEFICIO_FOLHA')==0)\
  .filter(col('TP_SITUACAO_FOLHA').isin(4, 7)) \
  .filter(col('IN_EXCLUSAO')==0)\
  .filter(col('ID_ERRO_FICHA').isNull())\
  .filter(dfScbaFichaFinanceira.TP_SOLICITACAO.isin(0, 1, 2)) \
  .select('ID_FICHA_FINANCEIRA', 'ID_PROCESSO', dfScbaFichaFinanceira.TP_SOLICITACAO, 'DT_PROCESSAMENTO', 'TP_BENEFICIO_FOLHA', dfScbaFolhaPagamento.DH_ULTIMA_ALTERACAO, 'TP_SITUACAO_FOLHA', 'TP_BENEFICIO')
#display(SQ_FICHA_FINANCEIRA_02)

##### Devolução do Recurso

In [0]:
dfScbaDevolucaoRecurso = spark.table('proddg.SCBA_DEVOLUCAO_RECURSO')
dfProcessoOcorrencia = spark.table('proddg.processo_ocorrencia')

In [0]:
SQ_DEVOLUCAO_RECURSO_04 = \
dfScbaRubricaFichaFinanceira \
  .join(broadcast(dfScbaDevolucaoRecurso), 'ID_RUBRICA_FICHA_FINANCEIRA', 'inner') \
  .join(dfProcessoOcorrencia, 'ID_OCORRENCIA', 'inner') \
  .filter(dfScbaRubricaFichaFinanceira.IN_EXCLUSAO==0) \
  .filter(dfProcessoOcorrencia.ST_OCORRENCIA==1) \
  .select('ID_FICHA_FINANCEIRA', dfScbaRubricaFichaFinanceira.ID_RUBRICA, 'AN_MES_REFERENCIA', dfScbaDevolucaoRecurso.ID_MOEDA, 'VL_DEVOLUCAO')\
  .withColumn('ID_RUBRICA_FICHA_FINANCEIRA', lit(None))\
  .withColumn('TP_NATUREZA_RUBRICA', lit('D'))\
  .withColumn('QT_RUBRICA', lit(1))\
  .withColumn('NR_EMPENHO', lit('Não Aplicável'))\
  .withColumn('NR_ORDEM_BANCARIA', lit('Não Aplicável'))\
  .withColumnRenamed('VL_DEVOLUCAO', 'VL_RUBRICA') \
  .select('ID_RUBRICA_FICHA_FINANCEIRA', 'ID_FICHA_FINANCEIRA', 'TP_NATUREZA_RUBRICA', 'ID_RUBRICA', 'AN_MES_REFERENCIA', dfScbaDevolucaoRecurso.ID_MOEDA, 'QT_RUBRICA', 'VL_RUBRICA', 'NR_EMPENHO', 'NR_ORDEM_BANCARIA')
#display(SQ_DEVOLUCAO_RECURSO_04)

##### Rubrica Ficha Financeira 06

In [0]:
dfScbaSolicitacaoRubrica = spark.table('proddg.SCBA_SOLICITACAO_RUBRICA')

In [0]:
SQ_RUBRICA_FICHA_FINANCEIRA_06 = \
dfScbaRubricaFichaFinanceira \
  .join(dfProcessoParcelaCronograma, 'ID_PARCELA_CRONOGRAMA', 'leftouter')\
  .join(dfScbaSolicitacaoRubrica, 'ID_RUBRICA_FICHA_FINANCEIRA', 'leftouter')\
  .filter(col('IN_EXCLUSAO')==0) \
  .filter(col('ST_RUBRICA').isin(1,3,4,5)==False) \
.select('ID_RUBRICA_FICHA_FINANCEIRA', 'ID_FICHA_FINANCEIRA', 'ID_RUBRICA', 'TP_NATUREZA_RUBRICA', 'AN_MES_REFERENCIA', 'VL_RUBRICA', 'ID_MOEDA', 'QT_RUBRICA', 'NR_EMPENHO', 'NR_ORDEM_BANCARIA')
#display(SQ_RUBRICA_FICHA_FINANCEIRA_06)

##### Rubrica Ficha Financeira 06 Devolução

In [0]:
dfProcessoParcelaCronogramaDevolucao = dfProcessoParcelaCronograma
dfScbaSolicitacaoRubricaDevolucao = dfScbaSolicitacaoRubrica
dfScbaRubricaFichaFinanceiraDevolucao = dfScbaRubricaFichaFinanceira

In [0]:
SQ_RUBRICA_FICHA_FINANCEIRA_06_DEVOLUCAO = \
dfScbaRubricaFichaFinanceiraDevolucao \
  .join(dfProcessoParcelaCronogramaDevolucao, 'ID_PARCELA_CRONOGRAMA', 'leftouter')\
  .join(dfScbaSolicitacaoRubricaDevolucao, dfScbaSolicitacaoRubricaDevolucao.ID_RUBRICA_FICHA_FINANCEIRA==col('ID_RUBRICA_FICHA_FINAN_CREDITO'), 'leftouter')\
  .filter(dfScbaRubricaFichaFinanceiraDevolucao.IN_EXCLUSAO==0) \
  .filter(coalesce(col('ST_RUBRICA'), lit(0)).isin(1,3,4,5)==False) \
  .filter(dfScbaRubricaFichaFinanceiraDevolucao.TP_NATUREZA_RUBRICA=='D') \
.select(dfScbaRubricaFichaFinanceiraDevolucao.ID_RUBRICA_FICHA_FINANCEIRA, 'ID_FICHA_FINANCEIRA', 'ID_RUBRICA', dfScbaRubricaFichaFinanceiraDevolucao.TP_NATUREZA_RUBRICA, 'AN_MES_REFERENCIA', 'VL_RUBRICA', 'ID_MOEDA', 'QT_RUBRICA', 'NR_EMPENHO', 'NR_ORDEM_BANCARIA')
#display(SQ_RUBRICA_FICHA_FINANCEIRA_06_DEVOLUCAO)

##### Union de Pagamento e devolução

In [0]:
JNR_RFF_FF_01 = \
  SQ_DEVOLUCAO_RECURSO_04.select('ID_RUBRICA_FICHA_FINANCEIRA', 'ID_FICHA_FINANCEIRA', 'TP_NATUREZA_RUBRICA', 'ID_RUBRICA', 'AN_MES_REFERENCIA', 'ID_MOEDA', 'QT_RUBRICA', 'VL_RUBRICA', 'NR_EMPENHO', 'NR_ORDEM_BANCARIA') \
  .union(SQ_RUBRICA_FICHA_FINANCEIRA_06.select('ID_RUBRICA_FICHA_FINANCEIRA', 'ID_FICHA_FINANCEIRA', 'TP_NATUREZA_RUBRICA', 'ID_RUBRICA', 'AN_MES_REFERENCIA', 'ID_MOEDA', 'QT_RUBRICA', 'VL_RUBRICA', 'NR_EMPENHO', 'NR_ORDEM_BANCARIA')) \
  .union(SQ_RUBRICA_FICHA_FINANCEIRA_06_DEVOLUCAO.select('ID_RUBRICA_FICHA_FINANCEIRA', 'ID_FICHA_FINANCEIRA', 'TP_NATUREZA_RUBRICA', 'ID_RUBRICA', 'AN_MES_REFERENCIA', 'ID_MOEDA', 'QT_RUBRICA', 'VL_RUBRICA', 'NR_EMPENHO', 'NR_ORDEM_BANCARIA')) \
  .join(SQ_FICHA_FINANCEIRA_02, 'ID_FICHA_FINANCEIRA', 'inner') \
  .cache() \
  .select('ID_PROCESSO', 'DT_PROCESSAMENTO', 'TP_SOLICITACAO', 'ID_RUBRICA', 'AN_MES_REFERENCIA', 	'ID_MOEDA', 'QT_RUBRICA', 'TP_NATUREZA_RUBRICA', 'VL_RUBRICA', 'ID_FICHA_FINANCEIRA', 'ID_RUBRICA_FICHA_FINANCEIRA', 'ID_FICHA_FINANCEIRA', 'NR_EMPENHO', 'NR_ORDEM_BANCARIA', 'TP_BENEFICIO_FOLHA', 'DH_ULTIMA_ALTERACAO', 'TP_SITUACAO_FOLHA') \
  .withColumn('TP_SOLICITACAO', when(col('TP_NATUREZA_RUBRICA')=='D', lit(4)).otherwise(col('TP_SOLICITACAO'))) \
  .withColumn('NR_SOLICITACAO', \
              when(col('TP_SOLICITACAO')==0, lit(col('NR_EMPENHO'))) \
              .when(col('TP_SOLICITACAO')==1, lit(col('NR_ORDEM_BANCARIA'))) \
              .when(col('TP_SOLICITACAO')==0, lit(col('NR_ORDEM_BANCARIA'))) \
              .otherwise('Não Disponível')) \
  .withColumn('VL_RUBRICA', when(col('TP_NATUREZA_RUBRICA')=='R', lit(1)).otherwise(-1)*col('VL_RUBRICA')*col('QT_RUBRICA')) \
  .withColumn('DT_REFERENCIA', to_date(col('AN_MES_REFERENCIA'), 'yyyyMM')) \
  .select('ID_PROCESSO', 'ID_RUBRICA', 'TP_SOLICITACAO', 'NR_SOLICITACAO', 'DT_PROCESSAMENTO', 'DT_REFERENCIA', 'VL_RUBRICA', 'ID_MOEDA', 'TP_BENEFICIO_FOLHA', 'DH_ULTIMA_ALTERACAO', 'TP_SITUACAO_FOLHA')
#display(JNR_RFF_FF_01)

In [0]:
UNN_PAGAMENTOS_FINAL_02 = \
  JNR_RFF_FF_01 \
  .union(FIL_EMPENHO_01 \
    .withColumn('DH_ULTIMA_ALTERACAO', lit(None)) \
    .withColumn('TP_SITUACAO_FOLHA', lit(None)) \
    .withColumnRenamed('ID_MOEDA_PAGAMENTO', 'ID_MOEDA') \
    .withColumnRenamed('TP_BENEFICIO', 'TP_BENEFICIO_FOLHA') \
    .select('ID_PROCESSO', 'ID_RUBRICA', 'TP_SOLICITACAO', 'NR_SOLICITACAO', 'DT_PROCESSAMENTO', 'DT_REFERENCIA', 'VL_RUBRICA', 'ID_MOEDA', 'TP_BENEFICIO_FOLHA', 'DH_ULTIMA_ALTERACAO', 'TP_SITUACAO_FOLHA')) \
  .union(FIL_PAGAMENTO_02 \
    .withColumn('DH_ULTIMA_ALTERACAO', lit(None)) \
    .withColumn('TP_SITUACAO_FOLHA', lit(None)) \
    .withColumnRenamed('ID_MOEDA_PAGAMENTO', 'ID_MOEDA') \
    .withColumnRenamed('TP_BENEFICIO', 'TP_BENEFICIO_FOLHA') \
    .select('ID_PROCESSO', 'ID_RUBRICA', 'TP_SOLICITACAO', 'NR_SOLICITACAO', 'DT_PROCESSAMENTO', 'DT_REFERENCIA', 'VL_RUBRICA', 'ID_MOEDA', 'TP_BENEFICIO_FOLHA', 'DH_ULTIMA_ALTERACAO', 'TP_SITUACAO_FOLHA'))
#display(UNN_PAGAMENTOS_FINAL_02)

##### Cotação

In [0]:
dfDwdCotacao = spark.table('default.dwdcotacao')

In [0]:
LKP_DWD_COTACAO_26 = \
UNN_PAGAMENTOS_FINAL_02 \
  .withColumn('DT_COTACAO', col('DT_PROCESSAMENTO').cast('date')) \
  .join(dfDwdCotacao.withColumn('DT_COTACAO', col('dataHoraCotacao').cast('date')).withColumn('ID_MOEDA', col('idMoeda')), ['DT_COTACAO', 'ID_MOEDA'], 'left') \
.withColumn('VL_COMPRA_BRL', col('cotacaoCompraBRL')) \
.withColumn('VL_COMPRA_USD', col('cotacaoCompraUSD')) \
.select('ID_PROCESSO', 'ID_RUBRICA', 'TP_SOLICITACAO', 'NR_SOLICITACAO', 'DT_PROCESSAMENTO', 'DT_REFERENCIA', 'VL_RUBRICA', 'ID_MOEDA', 'TP_BENEFICIO_FOLHA', 'DH_ULTIMA_ALTERACAO', 'TP_SITUACAO_FOLHA', 'VL_COMPRA_BRL', 'VL_COMPRA_USD')
#display(LKP_DWD_COTACAO_26)

##### Lookups de SK

In [0]:
dfDwdProcessoBolsa = spark.table('dw_dri.dwd_processo_bolsa').filter(col('IN_REGISTRO_ATIVO')=='S')
dfDwdProcessoAuxpe = spark.table('dw_dri.dwd_processo_auxpe').filter(col('IN_REGISTRO_ATIVO')=='S')
dfDwdRubrica = spark.table('dw_dri.dwd_rubrica').filter(col('IN_REGISTRO_ATIVO')=='S')
dfDwdTipoSolicitacao = spark.table('dw_dri.dwd_tipo_solicitacao').filter(col('IN_REGISTRO_ATIVO')=='S')
dfDwdMoeda = spark.table('dw_dri.dwd_moeda').filter(col('IN_REGISTRO_ATIVO')=='S')

In [0]:
EXP_BUSCA_SK_PAG_07 = \
LKP_DWD_COTACAO_26 \
  .withColumn('ID_RUBRICA', when(col('ID_RUBRICA').isNull(), -1).otherwise(col('ID_RUBRICA'))) \
  .withColumn('ID_TIPO_SOLICITACAO', when(col('TP_SOLICITACAO').isNull(), -1).otherwise(col('TP_SOLICITACAO'))) \
  .withColumn('ID_MOEDA', when(col('ID_MOEDA').isNull(), -1).otherwise(col('ID_MOEDA'))) \
  .join(dfDwdProcessoBolsa.select(col('ID_PROCESSO_BOLSA').alias('ID_PROCESSO'), 'SK_PROCESSO_BOLSA'), 'ID_PROCESSO', 'left') \
  .join(dfDwdProcessoAuxpe.select(col('ID_PROCESSO_AUXPE').alias('ID_PROCESSO'), 'SK_PROCESSO_AUXPE'), 'ID_PROCESSO', 'left') \
  .join(dfDwdRubrica.select('ID_RUBRICA', 'SK_RUBRICA'), 'ID_RUBRICA', 'left') \
  .join(dfDwdTipoSolicitacao.select('SK_TIPO_SOLICITACAO', 'ID_TIPO_SOLICITACAO'), 'ID_TIPO_SOLICITACAO', 'left') \
  .join(dfDwdMoeda.select('SK_MOEDA', 'ID_MOEDA'), 'ID_MOEDA', 'left') \
  .withColumn('SK_PROCESSO_BOLSA', when((col('TP_BENEFICIO_FOLHA').isin(1, 2) | col('SK_PROCESSO_BOLSA').isNotNull()), col('SK_PROCESSO_BOLSA')) \
                                  .when(col('TP_BENEFICIO_FOLHA').isin(1, 2)==False, -2)) \
  .withColumn('SK_PROCESSO_BOLSA', when(col('SK_PROCESSO_BOLSA').isNull(), -1).otherwise(col('SK_PROCESSO_BOLSA'))) \
  .withColumn('SK_PROCESSO_AUXPE', when((col('TP_BENEFICIO_FOLHA').isin(0) | col('SK_PROCESSO_AUXPE').isNotNull()), col('SK_PROCESSO_AUXPE')) \
                                  .when(col('TP_BENEFICIO_FOLHA').isin(0)==False, -2)) \
  .withColumn('SK_PROCESSO_AUXPE', when(col('SK_PROCESSO_AUXPE').isNull(),-1).otherwise(col('SK_PROCESSO_AUXPE'))) \
  .withColumn('SK_RUBRICA', when(col('SK_RUBRICA').isNull(), -1).otherwise(col('SK_RUBRICA'))) \
  .withColumn('SK_TIPO_SOLICITACAO', when(col('SK_TIPO_SOLICITACAO').isNull(), -1).otherwise(col('SK_TIPO_SOLICITACAO'))) \
  .withColumn('NR_SOLICITACAO', when(col('NR_SOLICITACAO').isNull(), 'Não Disponível').otherwise(col('NR_SOLICITACAO'))) \
  .withColumn('SK_MOEDA', when(col('SK_MOEDA').isNull(), -1).otherwise(col('SK_MOEDA'))) \
  .withColumn('SK_DATA_PAGAMENTO', when(col('DT_PROCESSAMENTO').isNull(), -1).otherwise(date_format('DT_PROCESSAMENTO', 'yyyyMMdd').cast('integer'))) \
  .withColumn('SK_DATA_REFERENCIA', when(col('DT_REFERENCIA').isNull(), -1).otherwise(date_format('DT_REFERENCIA', 'yyyyMMdd').cast('integer'))) \
  .withColumn('SK_DATA_ORDENACAO', when(col('DH_ULTIMA_ALTERACAO').isNull(), -1).otherwise(date_format('DH_ULTIMA_ALTERACAO', 'yyyyMMdd').cast('integer'))) \
  .withColumn('VL_PAGAMENTO_ORIGEM', col('VL_RUBRICA')) \
  .withColumn('VL_PAGAMENTO_USD', round((when(col('VL_COMPRA_USD').isNull(), 1).otherwise(col('VL_COMPRA_USD')))*col('VL_PAGAMENTO_ORIGEM'), 2)) \
  .withColumn('VL_PAGAMENTO_BRL', round((when(col('VL_COMPRA_BRL').isNull(), 1).otherwise(col('VL_COMPRA_BRL')))*col('VL_PAGAMENTO_ORIGEM'), 2)) \
  .withColumn('SK_PAGAMENTO', monotonically_increasing_id())\
  .withColumn('DH_CARGA', lit(inicio_etl).cast('timestamp'))\
  .select('sk_pagamento', 'sk_processo_bolsa', 'sk_processo_auxpe', 'sk_rubrica', 'sk_tipo_solicitacao', 'nr_solicitacao', 'sk_moeda', 'sk_data_pagamento', 'sk_data_referencia', 'sk_data_ordenacao', 'vl_pagamento_origem', 'vl_pagamento_usd', 'vl_pagamento_brl', 'dh_carga')
#display(EXP_BUSCA_SK_PAG_07)

In [0]:
## Grava a tabela Hive
EXP_BUSCA_SK_PAG_07.sort(desc('DT_REFERENCIA')).write.saveAsTable('default.dwfPagamento', mode='overwrite')